<p style="text-align:center"> 
    <a href="https://www.linkedin.com/in/flavio-aguirre-12784a252/" target="_blank"> 
    <img src="../../assets/logo.png" width="200" alt="Flavio Aguirre Logo"> 
    </a>
</p>

# <h1 align="center"><font size="7"><strong>EDA with SQL</strong></font></h1>
<hr>

## Let's start by importing the necessary libraries

In [2]:
import sqlite3
import prettytable

prettytable.DEFAULT = 'DEFAULT'

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Load the SQL extension for Jupyter Notebook
%load_ext sql

## We connect to our database

In [3]:
try: 
    conn = sqlite3.connect("../../database/chicago_data.db") 
    print("✅ Connection successful") 
except Exception as e: 
        print(f"❌ Error connecting:{e}")

✅ Connection successful


We establish a connection between the `Magic SQL` module and the `chicago_data.db` database.

In [6]:
%sql sqlite:///../../database/chicago_data.db

Let's start this analysis with the total number of crimes recorded in the city of Chicago.

In [ ]:
# The total number of crimes can be understood as the number of records in the chicago_crime_data table.
%%sql 
SELECT COUNT(*) AS total_crimes 
FROM chicago_crime_data

 * sqlite:///../../database/chicago_data.db
Done.


total_crimes
533


If we wanted to see the average number of crimes per year we can use the following subquery

In [ ]:
%%sql
SELECT AVG(crimes_per_year) AS average_crimes_per_year
FROM (
    SELECT 
    strftime('%Y', date) AS year, 
    COUNT(*) AS crimes_per_year
    FROM chicago_crime_data
    GROUP BY year
) AS yearly_totals;

 * sqlite:///../../database/chicago_data.db
Done.


average_crimes_per_year
29.61111111111111


It is worth mentioning that the data is truncated and that is why the percentage is so small for such a large city that generally has a recorded average of 500,000 crimes per year.

<br>

We understand that the crime rate is due to the poverty rate in the city, so let's look at the names and numbers of the community areas with a per capita income of less than $11,000 per year.

In [15]:
%%sql
SELECT
community_area_name AS community_name,
community_area_number AS area_number,
per_capita_income AS per_capita_income
FROM chicago_socioeconomic_data
WHERE per_capita_income < 11000

 * sqlite:///../../database/chicago_data.db
Done.


community_name,area_number,per_capita_income
West Garfield Park,26.0,10934
South Lawndale,30.0,10402
Fuller Park,37.0,10432
Riverdale,54.0,8201


### Let's see how these crimes relate to young people.

Let's look at all the case numbers for crimes involving juveniles (children are not considered minors for the purposes of crime analysis).

In [ ]:
%%sql 
SELECT case_number 
FROM chicago_crime_data
WHERE description LIKE '%MINOR%'

 * sqlite:///../../database/chicago_data.db
Done.


CASE_NUMBER
HL266884
HK238408


All kidnapping crimes involving a child.

In [18]:
%%sql 
SELECT case_number 
FROM chicago_crime_data
WHERE primary_type = 'KIDNAPPING' AND description LIKE '%CHILD%'

 * sqlite:///../../database/chicago_data.db
Done.


CASE_NUMBER
HN144152


The types of crimes reported in schools. (No repetition)

In [19]:
%%sql
SELECT DISTINCT primary_type AS crime_type 
FROM chicago_crime_data
WHERE location_description LIKE'%SCHOOL%'

 * sqlite:///../../database/chicago_data.db
Done.


crime_type
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


Let's see how well schools are prepared for these types of situations. We list the types of schools along with the average safety score for each type.

In [ ]:
%%sql
# We renamed the column 'Elementary, Middle, or High School' to avoid conflicts with SQL syntax
ALTER TABLE chicago_public_schools_data
RENAME COLUMN 'Elementary, Middle, or High School' TO elementary_middle_or_high_school;

 * sqlite:///../../database/chicago_data.db
Done.


[]

In [29]:
%%sql
SELECT 
DISTINCT elementary_middle_or_high_school AS school_type, 
AVG(safety_score) AS Score_Safety_average_by_school_type
FROM chicago_public_schools_data
WHERE elementary_middle_or_high_school IN ('ES','HS','MS')
GROUP BY elementary_middle_or_high_school
ORDER BY elementary_middle_or_high_school

 * sqlite:///../../database/chicago_data.db
Done.


school_type,Score_Safety_average_by_school_type
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


We see that these scores are alarmingly low.

Let's look at the 5 community areas with the highest percentage of households below the poverty line.

In [31]:
%%sql
SELECT 
community_area_name,
percent_households_below_poverty
FROM chicago_socioeconomic_data
ORDER BY percent_households_below_poverty DESC 
LIMIT 5

 * sqlite:///../../database/chicago_data.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


The truth is that observing how these communities have poverty rates of more than 50% is a source of discussion, as it reflects not only an alarming economic reality but also a profound social debt that challenges us as a society to promote more equitable, inclusive, and sustainable policies.

Considering this, which area of ​​the community is most prone to crime?

In [36]:
%%sql
SELECT 
    c.community_area_number,
    s.community_area_name,
    COUNT(*) AS number_crimes
FROM chicago_crime_data AS c
JOIN chicago_socioeconomic_data AS s
    ON c.community_area_number = s.community_area_number
GROUP BY c.community_area_number, s.community_area_name
ORDER BY number_crimes DESC
LIMIT 1;


 * sqlite:///../../database/chicago_data.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,number_crimes
25.0,Austin,43


Let's see which community has the highest deprivation index or hardship index.

In [ ]:
%%sql
SELECT 
community_area_name AS community_area,
hardship_index
FROM chicago_socioeconomic_data
WHERE hardship_index IN 
(SELECT MAX(hardship_index) FROM chicago_socioeconomic_data)

 * sqlite:///../../database/chicago_data.db
Done.


community_area,HARDSHIP_INDEX
Riverdale,98.0


The shortages this community is going through are simply brutal...

We see then that Rivardale is the community with the highest deprivation index (we would say unsurpassed...) at 98%, while Austin is the area with the most recorded crimes, with a total of 43 crimes in our database.

This analysis integrated data on crime, education, and socioeconomic factors, revealing deep inequalities. While there is still room for exploration, we demonstrated how SQL can address complex, real-world challenges.

In [ ]:
# We finish by closing the connection to the database
try:
    conn.close()
    print("✅ Connection closed successfully")
except Exception as e:
    print(f"❌ Error closing connection: {e}")


✅ Connection closed successfully


<br>

<hr>

## Author

<a href="https://www.linkedin.com/in/flavio-aguirre-12784a252/">**Flavio Aguirre**</a>
<br>
<a href="https://coursera.org/share/e27ae5af81b56f99a2aa85289b7cdd04">***Data Scientist***</a>